# Project 1.) First Cloud Function

### Description : Post a cloud function that takes in a string of numbers and returns a json file that contains the the sum of all of the single digit numbers

#### Example : input ="12345"
#### output = 1+2+3+4+5 = 15
#### returns({"answer":15})

In [1]:
def sum_num(string):
    output = 0
    for char in string:
        output += int(char)
    return ({"answer":output})

In [2]:
string = "12345"
result = sum_num(string)
result

{'answer': 15}

In [1]:
import requests
import json

In [2]:
string = "12345"
url = "https://us-central1-cloud-automation-424717.cloudfunctions.net/Q1"
data = {"i": string}
headers = {'Content-Type': 'application/json'}

response = requests.post(url, headers=headers, data=json.dumps(data))


In [3]:
response.json()

{'answer': 15}

## 1.b.) Query your cloud function using requests for example input "012937", "2" and "9999999999999"

In [4]:
string1 = "012937"
data1 = {"i": string1}
response = requests.post(url, headers=headers, data=json.dumps(data1))
response.json()

{'answer': 22}

In [5]:
string2 = "2"
data2 = {"i": string2}
response = requests.post(url, headers=headers, data=json.dumps(data2))
response.json()

{'answer': 2}

In [6]:
string3 = "9999999999999"
data3 = {"i": string3}
response = requests.post(url, headers=headers, data=json.dumps(data3))
response.json()

{'answer': 117}

# Project 3.) 

### Description : Build some machine learning model using scikit learn and make it queriable using cloud functions

In [42]:
import google

In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score

data = load_digits()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

feature_importances = clf.feature_importances_

feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

top_n = 10  
top_features = feature_importance_df['feature'].head(top_n).values

print(f"Top features: {top_features}")

X_top_train = X_train[top_features]
X_top_test = X_test[top_features]

clf_top = RandomForestClassifier(n_estimators=100, random_state=42)
clf_top.fit(X_top_train, y_train)

y_pred_top = clf_top.predict(X_top_test)
accuracy_top = accuracy_score(y_test, y_pred_top)
print(f'Accuracy with top {top_n} features: {accuracy_top:.2f}')


Top features: ['pixel_2_5' 'pixel_4_4' 'pixel_5_3' 'pixel_3_2' 'pixel_5_2' 'pixel_2_4'
 'pixel_7_5' 'pixel_3_4' 'pixel_3_3' 'pixel_1_2']
Accuracy with top 10 features: 0.93


In [92]:
df.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [120]:
bucket_name = "digits_q3"

In [136]:
from google.cloud import storage
import os
from io import StringIO
import pandas as pd

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cred keys.json"


In [137]:
client = storage.Client()

In [138]:
import joblib
joblib.dump(clf_top, "RF.sav") 

['RF.sav']

In [139]:
bucket = client.get_bucket("digits_q3")
blob = bucket.blob("digits/handwritten_numbers.sav")
blob.upload_from_filename("RF.sav")

In [140]:
from io import BytesIO 

In [141]:
def load_rf_model(file_name):
    bucket_name = "digits_q3"
    source_blob = "digits/" + file_name
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cred keys.json"
    client = storage.Client()
    
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob)
    
    model_data = blob.download_as_bytes()
    
    model = joblib.load(BytesIO(model_data))
    return(model)

In [142]:
model = load_rf_model("handwritten_numbers.sav")

In [143]:
import warnings
import numpy as np

In [144]:
def digits(request_dictionary):
    try:
        model = load_rf_model("handwritten_numbers.sav")

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            
            dictionary = request_dictionary
            input_values = [dictionary.get(key, 0) for key in top_features]

            X = np.array([input_values])  
    
            prediction = model.predict(X)[0]
            y_pred_top = model.predict(X_top_test)
            accuracy = accuracy_score(y_test, y_pred_top)

            return {"prediction": int(prediction), "accuracy": accuracy}
    except Exception as e:
        return {"error": str(e)}


In [145]:
digits({
    "pixel_2_5": 1,  
    "pixel_4_4": 15, 
    "pixel_5_3": 0,  
    "pixel_3_2": 12,  
    "pixel_5_2": 12,  
    "pixel_2_4": 1, 
    "pixel_7_5": 4, 
    "pixel_3_4": 2, 
    "pixel_3_3": 1,  
    "pixel_1_2": 9   
})

{'prediction': 4, 'accuracy': 0.9277777777777778}

In [153]:
import google
import joblib
import pandas
import requests
import sklearn
from urllib.parse import parse_qs
from google.cloud import storage
import os
from io import StringIO
from joblib import load
from io import BytesIO


In [ ]:
#on cloud
def load_rf_model(file_name):
    bucket_name = "digits_q3"
    source_blob = "digits/" + file_name
    
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cred keys.json"
    client = storage.Client()
    
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob)
    
    model_data = blob.download_as_bytes()
    
    model = joblib.load(BytesIO(model_data))
    return(model)

def digits(request):
    try:
        model = load_rf_model("handwritten_numbers.sav")

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)

            print(request)
            print(type(request))
            print(dir(request))

            q_str = request.get_data().decode()
            print(q_str)
            print(type(q_str))
            
            dictionary = request_dictionary
            input_values = {k: v[0] for k, v in parse_qs(q_str).items()}

            X = np.array([input_values])  
    
            prediction = model.predict(X)[0]
            y_pred_top = model.predict(X_top_test)
            accuracy = accuracy_score(y_test, y_pred_top)

            return {"prediction": int(prediction), "accuracy": accuracy}
    except Exception as e:
        return {"error": str(e)}



In [162]:
import requests
import json

url = "https://us-central1-cloud-automation-424717.cloudfunctions.net/Q3"
data = {
    "pixel_2_5": 1,  
    "pixel_4_4": 15, 
    "pixel_5_3": 0,  
    "pixel_3_2": 12,  
    "pixel_5_2": 12,  
    "pixel_2_4": 1, 
    "pixel_7_5": 4, 
    "pixel_3_4": 2, 
    "pixel_3_3": 1,  
    "pixel_1_2": 9   
}
headers = {'Content-Type': 'application/json'}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())


JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import google
import joblib
import pandas
import requests
import sklearn
from urllib.parse import parse_qs
from google.cloud import storage
import os
from io import StringIO
from joblib import load
from io import BytesIO

In [ ]:
r = requests.post("https://us-central1-cloud-automation-424717.cloudfunctions.net/Q3",
                 {
    "pixel_2_5": 1,  
    "pixel_4_4": 15, 
    "pixel_5_3": 0,  
    "pixel_3_2": 12,  
    "pixel_5_2": 12,  
    "pixel_2_4": 1, 
    "pixel_7_5": 4, 
    "pixel_3_4": 2, 
    "pixel_3_3": 1,  
    "pixel_1_2": 9   
})

In [ ]:
text_pixel_2_5 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_2_5",
    disabled = False
)

text_pixel_4_4 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_4_4",
    disabled = False
)

text_pixel_5_3 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_5_3",
    disabled = False
)

text_pixel_3_2 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_3_2",
    disabled = False
)

text_pixel_5_2 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_5_2",
    disabled = False
)

text_pixel_2_4 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_2_4",
    disabled = False
)

text_pixel_7_5 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_7_5",
    disabled = False
)

text_pixel_3_4 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_3_4",
    disabled = False
)

text_pixel_3_3 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_3_3",
    disabled = False
)

text_pixel_1_2 = widgets.Text(
    value = int,
    placeholder = "Type any value between 1 - 16",
    description = "pixel_1_2",
    disabled = False
)




button = widgets.Button(description = "Click when you have entered inputs")


def identify_digits(button):
  r = requests.post("https://us-central1-cloud-automation-424717.cloudfunctions.net/Q3",
                 {
    "pixel_2_5": 1,  
    "pixel_4_4": 15, 
    "pixel_5_3": 0,  
    "pixel_3_2": 12,  
    "pixel_5_2": 12,  
    "pixel_2_4": 1, 
    "pixel_7_5": 4, 
    "pixel_3_4": 2, 
    "pixel_3_3": 1,  
    "pixel_1_2": 9})
  print(r.json())


button.on_click(identify_digits)
